In [1]:
using Toolips

In [2]:
function makefrom_string(s::AbstractString, ogtag::String = "p")
    comps = []
    start = findall("<$ogtag", s)
    stop = findall("</$ogtag>", s)
    println(s)
    println(start, stop)
    for count in 1:length(stop)
        range = start[count][1]:stop[count][1]
        name_start = findall("id=", s[range])[1][2] + 1
        name_end = findnext(" ", s[range], name_start)[2] - 1
        tag = s[range[1] + 1:findnext(" ", s, r[1])]
        push!(comps, s[name_start:name_end] => tag)
    end
 #   [comps = cc[Component(c[1], c[2])] for c in comps]
end

makefrom_string (generic function with 2 methods)

In [3]:
function cfromstring(c::Component, s::String)
    name = c.name
    tag = c.tag
    tagrange = findall("<$tag id=\"$name\"", s)[1]
    tagend = findnext(">", s, tagrange[2])[1] + 1
    unsplit_props = s[tagrange[2]:tagend]
    ps = split(unsplit_props, " ")
    pairs = Vector{Pair{Any, Any}}()
    for p in 1:length(ps)
        if ps[p] == "="
            ps[p] = replace(ps[p], " " => "")
            push!(pairs, string(ps[p - 1]) => replace(string(join([ps[p + 1]])),
             "\"" => ""))
        end
    end
    props = Dict(pairs)
    props["children"] = Vector()
    props["text"] = ""
    c = Component(name, tag, props)
    endtag = findnext("</$tag>", s, tagrange[2])
    if ~(contains(s[tagend:endtag[1]], "<$tag"))
        if ~(contains(s[tagend:endtag[1] - 1], "<"))
            c["text"] = s[tagend:endtag[1] - 1]
        else
            comps = makefrom_string(s[tagrange[2] + 2:endtag[1] - 2])
            [push!(c, comp) for comp in comps]
        end
    else
        n_tags = findall("<$tag", s[tagrange[2]:length(s)])
        tag_ends = findall("</$tag>", s[tagrange[2]:length(s)])
        pos = 1
        for i in 1:length(tag_ends)
            if i in 1:length(n_tags)
                if n_tags[i][1] > tag_ends[i][1]
                    pos = i
                end
            else
                pos = i
            end
        end
        endtag = tag_ends[pos]
        comps = makefrom_string(s[tagend:length])
        [push!(c, comp) for comp in comps]
    end
    return(c)
end

cfromstring (generic function with 1 method)

In [4]:
using EzXML

In [5]:
s = """
<div id="example" width = "250" align="center">

<p id="steve2" place="south">Hey</p>

</div>
"""

"<div id=\"example\" width = \"250\" align=\"center\">\n\n<p id=\"steve2\" place=\"south\">Hey</p>\n\n</div>\n"

In [83]:
function htmlcomponent(s::String)
    doc = parsehtml(s)
    ro = root(doc)
    rn = firstnode(ro)
    children = Vector{Servable}()
    for n in eachelement(ro)
        if haselement(n)
            for node in eachelement(n)
                child = htmlcomponent(string(node))
                push!(children, child)
            end
        end
        comp = createcomp(n)
        comp[:children] = children
        push!(children, comp)
    end
    properties = Dict()
    for property in eachattribute(ro)
        sc = replace(string(property), "\"" => "")
        sc = replace(sc, " " => "")
        scspl = split(sc, "=")
        proppair = string(scspl[1]) => string(scspl[2])
        push!(properties, proppair)
    end
    c = Component("main", string(ro.name), properties)
    c[:children] = children
    c
end

htmlcomponent (generic function with 1 method)

In [84]:
function createcomp(element)
    properties = Dict()
    for property in eachattribute(element)
        sc = replace(string(property), "\"" => "")
        sc = replace(sc, " " => "")
        scspl = split(sc, "=")
        proppair = string(scspl[1]) => string(scspl[2])
        push!(properties, proppair)
    end
    children = Dict()
        tag = string(element.name)
        properties[:text] = string(element.content)
        name = "undefined $tag"
        if "id" in keys(properties)
            name = properties["id"]
        end
    Component(name, tag, properties)
end

createcomp (generic function with 1 method)

In [ ]:
x = htmlcomponent(s)

In [ ]:
x[:children][1].name

In [ ]:
x[:children, 1]

In [ ]:
typeof(x)

In [ ]:
function children(x)
    prnt = "##### children \n"
    for c in x[:children]
        prnt = prnt * "|-- " * string(c) * " \n "
        for subc in c[:children]
            prnt = prnt * "   |---- " * string(subc) * " \n "
        end
    end
    prnt
end

In [ ]:
import Base: string

In [74]:
function string(c::Component)
    base = c.name
    properties = ": "
    for pair in c.properties
        key, val = pair[1], pair[2]
        if ~(key == :children)
            properties = properties * "  $key = $val  "
        end
    end
    base * properties
end

string (generic function with 25 methods)

In [75]:
string(x)

"main: "

In [76]:
function show(x::Component)
    prnt = children(x)
    header = "### " * string(x) * "\n"
    display("text/markdown", header * prnt)
end

show (generic function with 1 method)

In [78]:
show(x[:children][1])

### main: 
##### children 
|-- steve2:   place = south    id = steve2    text = Hey   
    |---- steve2:   place = south    id = steve2    text = Hey   
 

In [65]:
import Base: show, display
import Toolips: SpoofConnection

In [20]:
function show(c::Component)
    md = show_str(c)
    display("text/markdown", md)
end

show (generic function with 1 method)

In [21]:
function show_str(c::Component)
    name = c.name
    props = ""
    for prop in c.properties
        if ~(prop[1] == :children)
            props = props * string(string(prop[1]) *  " = " * string(prop[2]) * "\n")
        end
    end
    md = """### $name
    $props \n
        \n"""
end

show_str (generic function with 1 method)

In [30]:
display(c::Servable) = show(c)

display (generic function with 31 methods)

In [31]:
x

Component("main", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("main", "html", Dict{Any, Any}(:children => Servable[Component("main", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("main", "html", Dict{Any, Any}(:children => Servable[Component("steve2", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("steve2", "p", Dict{Any, Any}("place" => "south", :children => Servable[#= circular reference @-4 =#], "id" => "steve2", :text => "Hey"), Servable[]), Dict{Any, Any}("place" => "south", :children => Servable[#= circular reference @-3 =#], "id" => "steve2", :text => "Hey"), Servable[], "p")]), Servable[]), Dict{Any, Any}(:children => Servable[Component("steve2", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("steve2", "p", Dict{Any, Any}("place" => "south", :children => Servable[#= circular reference @-4 =#], "id" => "steve2", :text => "Hey"), Servable[]), Dict{Any, Any}("place" => "south", :children => Ser